# 1st Part - Processing the input

## Getting the required code and models

In [ ]:
!git clone https://github.com/FreshMag/omr-img2midi.git
!mv /content/omr-img2midi/* /content/
!rm -rf omr-img2midi
!mkdir models
%cd models
!wget https://grfia.dlsi.ua.es/primus/models/PrIMuS/Semantic-Model.zip
!unzip Semantic-Model.zip -d semantic
!rm Semantic-Model.zip
%cd ..
!mkdir outputs

## Getting the Java tools for conversion: `semantic → MIDI/MEI`

In [ ]:
!wget https://grfia.dlsi.ua.es/primus/primus_converter.tgz
!tar xzvf primus_converter.tgz
!mv primus_conversor/omr-3.0-SNAPSHOT.jar ./converter/omr-3.0-SNAPSHOT.jar
!rm -rf ./primus_conversor
!rm ./primus_converter.tgz

## Installing required libraries

In [ ]:
!pip install midi2audio
!pip install verovio

## Import of python libraries and setup

In [ ]:
from utils import EncodedSheet
from ctc_predict import CTC
import verovio
import cv2
from translator import SheetTranslator

model_path = "/content/models/semantic/semantic_model.meta"
input_image_path = "/content/data/input/"
vocabulary_path = "/content/data/vocabulary_semantic.txt"

outputs_path = "/content/outputs/"

sheet = EncodedSheet(vocabulary_path)
model = CTC(model_path)

## Utility Functions

In [ ]:
from IPython.display import SVG, display, Markdown
import os

def get_filename_with_ex(filename, extension):
  return filename[:filename.rfind(".")] + "." + extension

def show_svg_file(filename):
  display(SVG(filename=filename))

def show_markdown(markdown_str):
  display(Markdown(markdown_str))

def mei2svg(input_mei_file, output_svg_file):
  tk = verovio.toolkit()
  tk.loadFile(input_mei_file)

  # Setting of verovio layout options
  options = {
      "scale": 85,
      "adjustPageHeight": True,
      "backgroundColor": "white"  # White background color
  }

  tk.setOptions(options)
  tk.renderToSVGFile(output_svg_file, 1)

def del_temp_files(output_file_path, extensions=['mei', 'svg']):
  base_file_path = get_filename_with_ex(output_file_path, "")
  for ext in extensions:
    os.remove(base_file_path + ext)



## Usage of the model

In [ ]:
# Set the image name here
image_name = "test1.png"


final_input_path = input_image_path + image_name
predictions = model.predict(final_input_path)
sheet.add_from_predictions(predictions)

print("Symbols found:")
sheet.print_symbols()

### Generation of the MEI file

In [ ]:
output_mei_path = outputs_path + get_filename_with_ex(image_name, "mei")
translator = SheetTranslator(sheet)
print(f"Converting to ${output_mei_path}")
translator.translate(output_mei_path)

### Generation of MIDI file

In [ ]:
output_midi_path = outputs_path + get_filename_with_ex(image_name, "mid")
translator = SheetTranslator(sheet)
print(f"Converting to ${output_midi_path}")
translator.translate(output_midi_path)



---

# 2nd Part - Displaying the results

## I - MEI file


### Rendering MEI file into SVG

In [ ]:
output_svg_path = outputs_path + get_filename_with_ex(image_name, "svg")

# utility function that converts MEI file into SVG file
mei2svg(output_mei_path, output_svg_path)


### Show the result

In [ ]:
from google.colab.patches import cv2_imshow

show_markdown("## Actual Input")
cv2_imshow(cv2.imread(final_input_path))

show_markdown("## Generated Input")
show_svg_file(output_svg_path)

## II - MIDI file

### Installing the required packages

In [ ]:
!apt-get update -qq && apt-get install -qq fluidsynth fluid-soundfont-gm
!pip install fluidsynth pyFluidSynth pretty_midi

### Playing MIDI file

In [ ]:
import pretty_midi
from IPython.display import Audio

def playMidi(midi_file_path):
    # Load MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)

    # Synthesize MIDI data into audio waveform
    audio_data = midi_data.synthesize()

    # Return audio as playable object
    return Audio(audio_data, rate=44100)

playMidi(output_midi_path)

# Clean temporary files

In [ ]:
# del_temp_files(outputs_path + image_name)